In [76]:
import pickle
from random import random
from typing import Tuple

from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow import keras
import tensorflow as tf
from seqeval.metrics import f1_score, classification_report

In [3]:
tf.test.is_built_with_cuda()

False

In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3344151048611754872
 xla_global_id: -1]

In [4]:
tf.config.list_physical_devices('GPU')

[]

# 양방향 LSTM(Bi-directional LSTM)
LSTM은 시퀀스 내 현재 시점을 오로지 이전 시점의 정보들로만 학습하고 추론한다는 한계가 있다.
이 문제를 해결코자 시퀀스 현재 시점 전후를 모두 읽어들이도록 확장한 LSTM이 양방향 LSTM이다.
간단하게 h_t = f(forward_lstm, backward_lstm) 정도로 표현할 수 있다.


In [5]:
def get_sequence(length: int) -> Tuple:
    x = np.array([random() for _ in range(length)])
    limit = length / 4.0
    y = np.array([0 if elem < limit else 1 for elem in np.cumsum(x)])

    # 양방향 LSTM 레이어에서 요구하는 형태(n_dim = 3)로 전처리.
    x = x.reshape(1, length, 1)
    y = y.reshape(1, length, 1)

    return x, y

In [6]:
n_units = 20
n_timestpes = 100

In [8]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=5,
    mode="auto")

input_layer = keras.layers.Input(shape=(n_timestpes, 1))
bi_lstm_layer_fn = keras.layers.Bidirectional(
    keras.layers.LSTM(units=n_units,
                      return_sequences=True))(input_layer)
dense_layer_fn = keras.layers.TimeDistributed(
    keras.layers.Dense(1, activation="sigmoid"))(bi_lstm_layer_fn)
bi_lstm_model = keras.Model(inputs=input_layer, outputs=dense_layer_fn)

bi_lstm_model.compile(loss="binary_crossentropy",
                      optimizer="adam",
                      metrics=["accuracy"])

In [9]:
dataset_x, dataset_y = get_sequence(n_timestpes)
test_x, test_y = get_sequence(n_timestpes)

In [10]:
history = bi_lstm_model.fit(dataset_x, dataset_y, epochs=1000, callbacks=[early_stopping])

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6970 - accuracy: 0.4000
Epoch 2/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.6942 - accuracy: 0.4200
Epoch 3/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.6916 - accuracy: 0.5700
Epoch 4/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.6892 - accuracy: 0.6500
Epoch 5/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.6869 - accuracy: 0.6300
Epoch 6/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.6846 - accuracy: 0.6200
Epoch 7/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.6825 - accuracy: 0.6000
Epoch 8/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.6805 - accuracy: 0.6000
Epoch 9/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.6786 - accuracy: 0.6000
Epoch 10/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.6768 - accuracy: 0.6000
Epoch 11/10

In [11]:
pred_y = bi_lstm_model.predict(test_x)

In [12]:
pred_y.shape

(1, 100, 1)

In [13]:
threshold = 0.05  # 차이가 이 값 미만이면 맞춘 걸로 인정.
ans = np.abs(pred_y - test_y) < threshold
acc = np.sum(ans) / n_timestpes
print(acc)

0.64


# 개체명 인식(NER: Named Entity Recognition)
"오늘 먹은 것은 제육볶음이다."란 문장이 있다고 하면 여기서 "오늘"은 "날짜", "제육볶음"은 "음식"이란
범주로 파악할 수 있다. 여기서 "날짜", "음식" 등 범주나 유형 등을 '개체명'이라고 부른다. 텍스트 내에서
단어의 개체명을 파악하는 일을 '개체명 인식'이라고 부른다.

텍스트 내 단어에 개체명을 붙이기 위한 표기법으로 'BIO(beginning, Inside, Outside) 표기법'이 있다.
B는 개체명이 시작하는 단어에, I는 B 태깅된 단어 뒤에서 그 개체명에 속하는 단어에, O는 아무 것도 아닌
단어에 붙이는 태그다.
"'도미니크 공화국'은 카리브 해의 히스파니올라 섬 동쪽 약 5/8을 차지하고 있는 나라다(나무위키)."는
도미니크(B-country), 공화국(I-country), 카리브(B-sea), 해(I-sea), 히스파니올라(B-island),
섬(I-island) 등으로 태깅할 수 있다('B-country'등은 임의로 붙인 것).

여기 사용된 데이터셋은 [KoreanNERCorpus](https://github.com/machinereading/KoreanNERCorpus)에서 받음.

## 데이터 처리

In [9]:
def read_file(file):
    """데이터 파일을 파싱

    파일은
        1. 세미콜론으로 시작하는 원본 문장,
        2. 그 다음엔 달러 기호로 시작하는 BIO 태깅된 문장,
        3. 그 이후부터 다음 원본 문장 전까지는 각 토큰의 피처들의 나열(ID, token, POS, BIO)
    이렇게 세 종류의 줄로 이루어져 있다.

    :param file: BIO 토큰 정보를 가진 파일.
    :return: 각 줄의 모든 피처를 열거한 리스트: [feats_sentence1, feats_sentence2, ...]
    """

    # 어떤 라인인지 파악하기 위한 조건
    def is_original_line(cur_i, lines):
        if cur_i >= len(lines) - 1:
            return False
        else:
            cur_line, next_line = lines[cur_i], lines[cur_i + 1]
            cur_first_char = cur_line[0]
            next_first_char = next_line[0]
            return cur_first_char == ";" and next_first_char == '$'

    def is_ner_processed_line(cur_i, lines):
        cur_line = lines[cur_i]
        first_char = cur_line[0]
        return first_char == "$"

    def is_end_of_datapoint(line):
        first_char = line[0]
        return first_char == "\n"

    features_all_lines = []  # 파일 내 모든 문장의 토큰 피처들
    with open(file, 'r', encoding="utf-8") as fin:
        lines = fin.readlines()

        for i, line in enumerate(lines):
            if is_original_line(i, lines):
                features_a_line = []
            elif is_ner_processed_line(i, lines):
                continue
            elif is_end_of_datapoint(line):
                features_all_lines.append(features_a_line)
            else:
                features_a_line.append(line.split())
    return features_all_lines

In [10]:
corpus = read_file("data/corpus/korean_ner/train.txt")

In [11]:
corpus[:5]

[[['1', '한편', 'NNG', 'O'],
  ['1', ',', 'SP', 'O'],
  ['2', 'AFC', 'SL', 'O'],
  ['2', '챔피언스', 'NNG', 'O'],
  ['2', '리그', 'NNG', 'O'],
  ['3', 'E', 'SL', 'B_OG'],
  ['3', '조', 'NNG', 'I'],
  ['3', '에', 'JKB', 'O'],
  ['4', '속하', 'VV', 'O'],
  ['4', 'ㄴ', 'ETM', 'O'],
  ['5', '포항', 'NNP', 'O'],
  ['6', '역시', 'MAJ', 'O'],
  ['7', '대회', 'NNG', 'O'],
  ['8', '8강', 'NNG', 'O'],
  ['9', '진출', 'NNG', 'O'],
  ['9', '이', 'JKS', 'O'],
  ['10', '불투명', 'NNG', 'O'],
  ['10', '하', 'VV', 'O'],
  ['10', '다', 'EC', 'O'],
  ['11', '.', 'SF', 'O']],
 [['1', '2003', 'SN', 'B_DT'],
  ['1', '년', 'NNB', 'I'],
  ['2', '6', 'SN', 'I'],
  ['2', '월', 'NNB', 'I'],
  ['3', '14', 'SN', 'I'],
  ['3', '일', 'NNB', 'I'],
  ['4', '사직', 'NNG', 'O'],
  ['5', '두산', 'NNP', 'O'],
  ['5', '전', 'NNG', 'O'],
  ['6', '이후', 'NNG', 'O'],
  ['7', '박명환', 'NNP', 'B_PS'],
  ['7', '에게', 'JKB', 'O'],
  ['8', '당하', 'VV', 'O'],
  ['8', '았', 'EP', 'O'],
  ['8', '던', 'ETM', 'O'],
  ['9', '10', 'SN', 'O'],
  ['9', '연패', 'NNG', 'O'],
  ['10', 

In [12]:
# 문장별로 토큰(입력)과 BIO 태그(정답)를 따로 수집한다.
tokens_all_sentences, bios_all_sentences = [], []
for tags_a_sentence in corpus:
    tokens_a_sentence, bios_a_sentence = [], []

    for features in tags_a_sentence:
        token, bio = features[1], features[3]
        tokens_a_sentence.append(token)
        bios_a_sentence.append(bio)

    tokens_all_sentences.append(tokens_a_sentence)
    bios_all_sentences.append(bios_a_sentence)

In [13]:
sentence_id = 0

tokens = tokens_all_sentences[sentence_id]
bios = bios_all_sentences[sentence_id]
n_samples = 10
print(f"첫번째 문장의 토큰-BIO 태그 샘플들 {n_samples}개:")
for t, b in zip(tokens[:n_samples], bios[:n_samples]):
    print(t, b)

첫번째 문장의 토큰-BIO 태그 샘플들 10개:
한편 O
, O
AFC O
챔피언스 O
리그 O
E B_OG
조 I
에 O
속하 O
ㄴ O


In [14]:
# 결과를 보면 BIO 중 O가 대다수인 편향이 있음을 알 수 있다. 따라서 학습된 모델은 정확도보다
# F-1 점수로 검증함이 더 공정하다.
def rate_category(cat, data):
    """

    :param cat: {BIO} 중 하나. 지금은 O에만 작동한다.
    :param data: 원소는 cat에 속하는 글자 하나로만 이루어져 있어야 한다.
    :return:
    """
    tensor = tf.ragged.constant(data)
    vector = tf.reshape(tensor, [-1])
    len_vector = tf.size(vector)
    count = tf.reduce_sum(tf.cast(tf.equal(cat, vector), tf.int32))
    return count / len_vector


count_O = rate_category('O', bios_all_sentences)

# rate_O = count_O / count_all
print(f"num('O')/num('BIO') = {count_O:3}")

num('O')/num('BIO') = 0.8567379285838244


In [15]:
# if given, it will be added to word_index and used to replace out-of-vocabulary
# words during text_to_sequence calls
token_tokenizer = keras.preprocessing.text.Tokenizer(oov_token="OOV")
token_tokenizer.fit_on_texts(tokens_all_sentences)

bio_tokenizer = keras.preprocessing.text.Tokenizer(lower=False)  # 토큰은 그대로 둔다.
bio_tokenizer.fit_on_texts(bios_all_sentences)

In [73]:
# 모델 추론 결과를 다시 읽을 수 있는 형태로 되돌리기 위해 매핑 저장.
token_index_word = token_tokenizer.index_word
tag_index_word = bio_tokenizer.index_word
tag_index_word[0] = 'PAD'  # `0` is a reserved index that won't be assigned to any word.

token_space_size = len(token_index_word) + 1
bio_space_size = len(tag_index_word) + 1

In [17]:
dataset_x = token_tokenizer.texts_to_sequences(tokens_all_sentences)
dataset_y = bio_tokenizer.texts_to_sequences(bios_all_sentences)

In [35]:
# 이 길이로 시퀀스를 고정하고, 임베딩 레이어의 `input_length` 인자로 준다.
maxlen = max(map(lambda token_sequence: len(token_sequence), dataset_x))
padded_dataset_x = keras.preprocessing.sequence.pad_sequences(
    dataset_x,
    maxlen=maxlen,  # maxlen 주어지지 않으면 자동으로 최장 시퀀스 길이를 따름.
    padding="post")
padded_dataset_y = keras.preprocessing.sequence.pad_sequences(
    dataset_y,
    maxlen=maxlen,
    padding="post")

In [36]:
train_x, test_x, train_y, test_y = train_test_split(
    padded_dataset_x,
    padded_dataset_y,
    test_size=.2)

In [37]:
one_hot_train_y = tf.keras.utils.to_categorical(train_y, num_classes=bio_space_size)
one_hot_test_y = tf.keras.utils.to_categorical(test_y, num_classes=token_space_size)

In [38]:
one_hot_train_y.shape

(2844, 168, 8)

In [39]:
print(train_x.shape)  # len_dataset, len_sequence
print(one_hot_train_y.shape)

(2844, 168)
(2844, 168, 8)


In [40]:
trainset = tf.data.Dataset.from_tensor_slices((train_x, one_hot_train_y)).batch(128)
testset = tf.data.Dataset.from_tensor_slices((test_x, one_hot_test_y)).batch(128)

## 모델 빌드

In [5]:
bi_lstm_model_dir = "models/bi_lstm_for_ner"

In [29]:
input_layer = keras.layers.Input(shape=[maxlen, ])
embedding_layer_fn = keras.layers.Embedding(
    input_dim=token_space_size,
    output_dim=30,
    mask_zero=True,  # 시퀀스를 0으로 패딩했다면 True로 값을 설정해 그 0이 패딩임을 고지.
    input_length=maxlen)  # Length of input sequences
bi_lstm_layer_fn = keras.layers.Bidirectional(
    tf.keras.layers.LSTM(
        200,  # 레이어 내 노드 수.
        return_sequences=True,  # 정방향과 역방향 출력을 하나의 시퀀스로 묶는다.
        dropout=.5,
        # T면 [timesteps, batch, feature] F면 [batch, timesteps, feature]의 입력
        # 형상으로 True일 떄가 연산 효율은 더 좋다(기본값 False).
        # time_major= True
        recurrent_dropout=.25))
# '어느 개체명이 가장 가능성 높은가?'를 마지막 밀집층으로 보여준다.
dense_layer_fn = keras.layers.TimeDistributed(
    keras.layers.Dense(bio_space_size, activation="softmax"))

embedding_layer = embedding_layer_fn(input_layer)
bi_lstm_layer = bi_lstm_layer_fn(embedding_layer)
dense_layer = dense_layer_fn(bi_lstm_layer)

bi_lstm_model = keras.Model(inputs=input_layer, outputs=dense_layer)
bi_lstm_model.compile(loss="categorical_crossentropy",
                      optimizer="adam",
                      metrics=["accuracy"])

In [30]:
# 오래 걸려서 GPU 달린 서버에서 실행(레거시 옵티마이저가 아님에 주의!)
bi_lstm_history = bi_lstm_model.fit(trainset,  # train_x, one_hot_train_y,
                                    epochs=1000,
                                    callbacks=[early_stopping])

Epoch 1/1000
23/23 [==============================] - 41s 2s/step - loss: 0.2412 - accuracy: 0.8261
Epoch 2/1000
23/23 [==============================] - 48s 2s/step - loss: 0.1303 - accuracy: 0.8572
Epoch 3/1000
23/23 [==============================] - 52s 2s/step - loss: 0.1243 - accuracy: 0.8572
Epoch 4/1000
23/23 [==============================] - 53s 2s/step - loss: 0.1190 - accuracy: 0.8572
Epoch 5/1000
23/23 [==============================] - 55s 2s/step - loss: 0.1114 - accuracy: 0.8566
Epoch 6/1000
23/23 [==============================] - 57s 2s/step - loss: 0.0958 - accuracy: 0.8635
Epoch 7/1000
23/23 [==============================] - 58s 3s/step - loss: 0.0800 - accuracy: 0.8770
Epoch 8/1000
23/23 [==============================] - 60s 3s/step - loss: 0.0694 - accuracy: 0.8856
Epoch 9/1000
23/23 [==============================] - 60s 3s/step - loss: 0.0631 - accuracy: 0.8921
Epoch 10/1000
23/23 [==============================] - 60s 3s/step - loss: 0.0590 - accuracy: 0.8991

In [31]:
bi_lstm_model.save(bi_lstm_model_dir)

INFO:tensorflow:Assets written to: models/bi_lstm_for_ner\assets


### 입력 형상 생각하기 귀찮으면...

In [31]:
another_model = keras.Sequential()

another_model.add(embedding_layer_fn)
another_model.add(bi_lstm_layer_fn)
another_model.add(dense_layer_fn)
another_model.compile(loss="categorical_crossentropy",
                      optimizer=keras.optimizers.legacy.Adam(),
                      metrics=["accuracy"])

In [32]:
another_model.fit(train_x,
                  one_hot_train_y,
                  batch_size=128,
                  epochs=1000,
                  callbacks=[early_stopping])

Epoch 1/1000


2023-01-18 18:19:46.985958: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

## 테스트셋으로 성능 측정

In [6]:
loaded_model = tf.saved_model.load(bi_lstm_model_dir)

Metal device set to: Apple M1 Pro


2023-01-18 21:25:43.065099: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-18 21:25:43.065119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [33]:
pred_fn = loaded_model.signatures["serving_default"]
test_x = test_x.astype(np.float32)  # 기본 타입 float32인 거 모델 빌드 때 까먹었다.

In [57]:
pred_y = pred_fn(input_3=test_x)  # 서명 잘못 설정했다...

In [64]:
# 임베딩된 상태라 그대로 읽기 어려우니 앞서 선언했던 `token_index_word`, `tag_index_word` 활용.
pred_y = pred_y["time_distributed_2"]

In [68]:
def sequences_to_tag(sequences, mapping_dict):
    """"one-hot 결과를 읽을 수 있는 문자로 변환한다.

    각 시퀀스는 토큰별 one-hot 추론 결과를 원소로 담고 있다. [num_seq, len_seq, len_one_hot]

    :param sequences: 추론된 원-핫 형식 태그 콜렉션. 추론 함수의 결과 텐서를 기대한다.
    :param mapping_dict: 인코딩된 태그값에 해당하는 원래 문자 태그의 매핑.
    :return: `mapping_dict`에 의해 문자로 변환된 시퀀스들.
    """
    tags_all_sequences = []
    for sequence in sequences:
        tags_a_sequence = []
        for one_hot_score in sequence:
            most_likely_index = np.argmax(one_hot_score)
            pred_tag = mapping_dict[most_likely_index].replace("PAD", "O")
            tags_a_sequence.append(pred_tag)
        tags_all_sequences.append(tags_a_sequence)

    return tags_all_sequences

In [72]:
pred_y.shape

TensorShape([711, 168, 8])

In [75]:
# 시간이 걸리므로 pickle 직렬화 보존하자.
decoded_pred_tags = sequences_to_tag(pred_y, tag_index_word)
decoded_test_tags = sequences_to_tag(test_y, tag_index_word)

In [77]:
with open("data/corpus/korean_ner/decoded_pred_tags.pickle", "wb") as fout:
     pickle.dump(decoded_pred_tags, fout)

with open("data/corpus/korean_ner/decoded_test_tags", "wb") as fout:
    pickle.dump(decoded_test_tags, fout)

In [78]:
report = classification_report(decoded_test_tags, decoded_pred_tags)
f1 = f1_score(decoded_test_tags, decoded_pred_tags)

/Users/hakjun/PycharmProjects/chatbot/venv/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/hakjun/PycharmProjects/chatbot/venv/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/hakjun/PycharmProjects/chatbot/venv/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/hakjun/PycharmProjects/chatbot/venv/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/hakjun/PycharmProjects/chatbot/venv/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_TI seems not

In [79]:
print(report)
print(f1)

              precision    recall  f1-score   support

           _       0.00      0.00      0.00         0
         _DT       0.00      0.00      0.00         0
         _LC       0.00      0.00      0.00         0
         _OG       0.00      0.00      0.00         0
         _PS       0.00      0.00      0.00         0
         _TI       0.00      0.00      0.00         0

   micro avg       0.00      0.00      0.00         0
   macro avg       0.00      0.00      0.00         0
weighted avg       0.00      0.00      0.00         0

0.0


In [85]:
sum([p == a for p, a in zip(decoded_pred_tags[0], decoded_test_tags[0])])

157